In [ ]:
!pip install yfinance==0.2.4
#!pip install pandas==1.3.3
#!pip install requests==2.26.0
!mamba install bs4==4.10.0 -y
#!pip install plotly==5.3.1
!pip install html5lib
!pip install --upgrade nbformat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 427.0 kB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.1.67
    Uninstalling yfinance-0.1.67:
      Successfully uninstalled yfinance-0.1.67


/bin/bash: line 1: mamba: command not found
ERROR: Operation cancelled by user
^C


In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [3]:
tesla = yf.Ticker("TSLA")

In [4]:
tesla_data = tesla.history(period="max")

Failed to get ticker 'TSLA' reason: Expecting value: line 1 column 1 (char 0)
- TSLA: No timezone found, symbol may be delisted


In [6]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,index,Date,Open,High,Low,Close,Adj Close,Volume


In [7]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data  = requests.get(url).text

In [8]:
soup = BeautifulSoup(html_data,"html5lib")

In [9]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

for table in soup.find_all('table'):

    if ('Tesla Quarterly Revenue' in table.find('th').text):
        rows = table.find_all('tr')

        for row in rows:
            col = row.find_all('td')

            if col != []:
                date = col[0].text
                revenue = col[1].text.replace(',','').replace('$','')

                tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

In [10]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

In [11]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [12]:
tesla_revenue.tail()

,Date,Revenue


In [13]:
GameStop = yf.Ticker("GME")

In [14]:
gme_data = GameStop.history(period="max")

Failed to get ticker 'GME' reason: Expecting value: line 1 column 1 (char 0)
- GME: No timezone found, symbol may be delisted


In [15]:
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume


In [16]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data  = requests.get(url).text

In [17]:
soup = BeautifulSoup(html_data,"html5lib")

In [19]:
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])
new_rows = []

for table in soup.find_all('table'):

    if ('GameStop Quarterly Revenue' in table.find('th').text):
        rows = table.find_all('tr')

        for row in rows:
            col = row.find_all('td')

            if col != []:
                date = col[0].text
                revenue = col[1].text.replace(',','').replace('$','')

                new_rows.append({"Date":date, "Revenue":revenue})

gme_revenue = pd.concat([gme_revenue, pd.DataFrame(new_rows)], ignore_index=True)

In [20]:
gme_revenue.tail()

,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709


In [21]:
pip install plotly

In [22]:
import plotly.graph_objects as go
tsla = yf.download('TSLA', start='2020-01-01', end='2023-01-01')
# Stock price chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=tsla.index, y=tsla['Close'], name='Stock Price'))

# Revenue chart
fig.add_trace(go.Bar(x=tesla_revenue['Date'], y=tesla_revenue['Revenue'], name='Revenue', yaxis='y2'))

# Update layout for dual y-axis
fig.update_layout(
    title='Tesla Stock Price and Revenue',
    xaxis_title='Date',
    yaxis_title='Stock Price',
    yaxis2=dict(
        title='Revenue',
        overlaying='y',
        side='right'
    )
)

fig.show()


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TSLA: No timezone found, symbol may be delisted


In [23]:
import plotly.graph_objects as go
gme = yf.download('GME', start='2020-01-01', end='2023-01-01')

# Stock price chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=gme.index, y=gme['Close'], name='Stock Price'))

# Revenue chart
fig.add_trace(go.Bar(x=gme_revenue['Date'], y=gme_revenue['Revenue'], name='Revenue', yaxis='y2'))

# Update layout for dual y-axis
fig.update_layout(
    title='GameStop Stock Price and Revenue',
    xaxis_title='Date',
    yaxis_title='Stock Price',
    yaxis2=dict(
        title='Revenue',
        overlaying='y',
        side='right'
    )
)

fig.show()


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- GME: No timezone found, symbol may be delisted


In [40]:
import yfinance as yf
import pandas as pd

# Fetch TSLA data
tsla_ticker = yf.Ticker("TSLA")
tsla_data = tsla_ticker.history(period="max")

# Fetch GME data
gme_ticker = yf.Ticker("GME")
gme_data = gme_ticker.history(period="max")

# Check if data is successfully downloaded
if tsla_data.empty or gme_data.empty:
    print("Error: Could not fetch TSLA or GME stock data using yf.Ticker().history(). Please check your internet connection or try again later.")
else:
    print("TSLA Data Head:")
    display(tsla_data.head())

    print("\nGME Data Head:")
    display(gme_data.head())

Failed to get ticker 'TSLA' reason: Expecting value: line 1 column 1 (char 0)
- TSLA: No timezone found, symbol may be delisted
Failed to get ticker 'GME' reason: Expecting value: line 1 column 1 (char 0)
- GME: No timezone found, symbol may be delisted
Error: Could not fetch TSLA or GME stock data using yf.Ticker().history(). Please check your internet connection or try again later.


In [29]:
!pip install pandas_datareader

In [30]:
import pandas_datareader.data as web
import datetime as dt

# Define the start and end dates for the data
start_date = dt.datetime(2020, 1, 1)
end_date = dt.datetime(2023, 1, 1)

# Fetch TSLA data using pandas_datareader
try:
    tsla_data = web.DataReader('TSLA', 'stooq', start=start_date, end=end_date)
    print("TSLA Data Head:")
    display(tsla_data.head())
except Exception as e:
    print(f"Error fetching TSLA data: {e}")
    tsla_data = pd.DataFrame() # Assign empty DataFrame in case of error

# Fetch GME data using pandas_datareader
try:
    gme_data = web.DataReader('GME', 'stooq', start=start_date, end=end_date)
    print("\nGME Data Head:")
    display(gme_data.head())
except Exception as e:
    print(f"Error fetching GME data: {e}")
    gme_data = pd.DataFrame() # Assign empty DataFrame in case of error

TSLA Data Head:


,Open,High,Low,Close,Volume
Date,,,,,
2022-12-30,119.950,124.4800,119.750,123.18,157777339
2022-12-29,120.385,123.5700,117.495,121.82,221923313
2022-12-28,110.350,116.2700,108.240,112.71,221070537
2022-12-27,117.495,119.6700,108.760,109.10,208643444
2022-12-23,126.370,128.6173,121.020,123.15,166989688



GME Data Head:


,Open,High,Low,Close,Volume
Date,,,,,
2022-12-30,17.95,18.575,17.90,18.46,2669548
2022-12-29,18.12,18.570,17.75,18.33,3442757
2022-12-28,18.29,18.700,16.98,17.92,5671992
2022-12-27,19.91,19.990,18.00,18.20,4233045
2022-12-23,20.00,20.630,19.65,20.08,3174827


In [31]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add TSLA stock price trace
fig.add_trace(go.Scatter(x=tsla_data.index, y=tsla_data['Close'], mode='lines', name='TSLA Stock Price'))

# Add GME stock price trace
fig.add_trace(go.Scatter(x=gme_data.index, y=gme_data['Close'], mode='lines', name='GME Stock Price'))

# Update layout
fig.update_layout(
    title='Tesla vs GameStop Stock Price Comparison (2020-2023)',
    xaxis_title='Date',
    yaxis_title='Stock Price (USD)',
    legend_title='Company',
    hovermode='x unified' # Improves hover experience
)

# Show the plot
fig.show()

In [38]:
# We have gme_data from pandas_datareader and gme_revenue from web scraping
# Let's create the combined plot for GameStop

# Ensure Date columns are in datetime format for plotting
gme_data['Date'] = pd.to_datetime(gme_data.index)
gme_revenue['Date'] = pd.to_datetime(gme_revenue['Date'])

# Use the make_graph function defined earlier
# Align the dataframes by date if necessary for accurate plotting
# For simplicity, let's use the full data ranges we have and let Plotly handle the alignment

make_graph(gme_data, gme_revenue, 'GameStop (GME)')

/tmp/ipython-input-2-2068038883.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/tmp/ipython-input-2-2068038883.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

